Main crRNA features to consider:
1. Specificity (within target organism)
2. Isoform prevalence
3. Proximity to the N-terminus
4. Absence of alternative start codons downstream of the cut site
5. Avoid exon-exon boundaries
6. Not spanning know SNP sites

Other features that might stem from the preference of the Csm:
7. No low complexity regions
8. *Balanced GC content*
9. No self-complementarity
10. No target mRNA secondary structures within the binding region
11. No crRNA secondary structures

Use magic for inline interactive plotting.

In [1]:
%matplotlib notebook

Do the imports.

In [2]:
import gffutils
import os
import numpy as np
import matplotlib.pyplot as plt
import subprocess
import sys
from multiprocessing import Pool
from Bio import SeqIO
from collections import defaultdict, Counter
import pickle
from decimal import *
import pandas as pd
from pybedtools import BedTool

---

Create or load the GFF annotation database

In [3]:
db_file = 'danRer.GRCz10.90.db'
if not os.path.exists(db_file):
    db = gffutils.create_db("Danio_rerio.GRCz10.90.gtf",
                            dbfn="danRer.GRCz10.90.db",
                            id_spec={'gene': 'gene_id', 'transcript': "transcript_id"},
                            disable_infer_genes=True,
                            disable_infer_transcripts=True,
                            force=False)
else:
    db = gffutils.FeatureDB(db_file)

Create an indexed fasta database for faster access with SeqIO.

In [4]:
fa_db = SeqIO.index_db('danRer.GRCz10.dna_sm.toplevel.idx', 'Danio_rerio.GRCz10.dna_sm.toplevel.fa', 'fasta')

Load the variation information.

In [5]:
gvf_fn=('danio_rerio.gvf.gz')

Define functions

In [215]:
def revcomp(seq):
    # reverse complement sequence
    
    assert isinstance(seq, str)
    # check if sequence is RNA
    if seq.upper().find('U') == -1:
        # DNA
        comp_b = {'A': 'T', 'T': 'A', 'G': 'C', 'C': 'G', 'N': 'N', 
                  'a': 't', 't': 'a', 'g': 'c', 'c': 'g', 'n': 'n'}
    else:
        # RNA
        comp_b = {'A': 'U', 'U': 'A', 'G': 'C', 'C': 'G', 'N': 'N', 
                  'a': 'u', 'u': 'a', 'g': 'c', 'c': 'g', 'n': 'n'}
    rc_seq = ''.join([comp_b[b] for b in seq][::-1])
    return rc_seq


def gc(seq):
    # return gc content percentage
    
    assert isinstance(seq, str)
    seq = seq.upper()
    gc_content = 100 * Decimal(seq.count("G") + seq.count("C")) / len(seq)
    return gc_content


def transcribe(seq):
    # transcribe sequence- change T to U
    
    assert isinstance(seq, str)
    trans_seq = seq.replace('T', 'U').replace('t', 'u')
    return trans_seq


def revtranscribe(seq):
    # reverse transcribe sequence- change U to T
    
    assert isinstance(seq, str)
    trans_seq = seq.replace('U', 'T').replace('u', 't')
    return trans_seq


def get_cov(database,gene):
    # Create genomic sequence coverage by CDS (without STOP codon).
    
    # get the length of the gene (1 based coords)
    gene_length = gene.end - gene.start + 1
    print('{} loci lenght: {!s}.'.format(gene.id, gene_length))
    # create a numpy array of zeros with the length of the gene
    gene_cov = np.zeros(gene_length, dtype=int)
    for cds in db.children(gene, featuretype='CDS', order_by='start'):
            # get relative exon coords in 0-based space
            rel_start = cds.start - gene.start
            rel_end = cds.end - gene.start
            gene_cov[rel_start:rel_end + 1] += 1
    print('Maximal genomic sequence coverage by CDS: {!s}.'.format(gene_cov.max()))
    return gene_cov


def get_int(gene, genomic_coverage):
    # get the genomic intervals (for sequene retrieval)
    
    gen_int = []
    cov = 0
    start = 0 
    for i,c in enumerate(genomic_coverage):
        # adjust the genomic coords so they would be 0 based, non-inclusive (*similar to BED*)
        if cov == 0 and c!=0:
            # start cov equals zero and now it changed- store start position and coverage
            start = i
            cov = c
        elif cov != 0 and c!= cov:
            # coverage was not zero but it changed- store the previous interval , new start position and new coverage
            gen_int += [[gene.chrom, start + gene.start - 1, i + gene.start - 1, cov]]
            start = i
            cov = c
        # else if the coverage is the same- do nothing
    # in the end store also the coverage of the last segment if the coverage was not equal to 0
    if cov != 0:
        gen_int += [[gene.chrom, start + gene.start - 1, i + gene.start, cov]]
    i_cnt = Counter()
    for e in gen_int:
        i_cnt[e[3]] += 1
    print('{!s} CDS coverage intervals (coverage, count): {!s}.'.format(len(gen_int), i_cnt.most_common()))
    return gen_int


def sub_var(intervals, variation_fn):
    '''
    Subtract variable sequences from the genomic intervals
    
    For future enhancemnt consider soft masking the whole genome beforehand or splitting the GVF file
    on chromosomes for speed improvements.
    '''
    
    # make some basic checks
    assert os.path.exists(variation_fn)
    assert variation_fn.endswith(('gvf', 'gvf.gz'))
    
    gvf_sorted_fn = 'sorted.gvf'.join(variation_fn.split('gvf'))
    # sort the variation database for faster processing and store for further use
    if not os.path.exists(gvf_sorted_fn):
        print('Sorting and saving the new GVF file as {}.'.format(gvf_sorted_fn))
        gvf_bt = BedTool(gvf_fn).sort().saveas(gvf_sorted_fn)
    else:
        gvf_bt = BedTool(gvf_sorted_fn)

    print('Subtracting variation from {}.'.format(gvf_bt[1]['Dbxref'].split(':')[0]))

    # make sure the format is right
    for e in intervals:
        assert len(e) == 4
    # perform genome aritmethics
    gi_gv = BedTool(('\t'.join([c, str(s), str(e), '.', str(cov)]) for c,s,e,cov in intervals))\
    .saveas()\
    .subtract(gvf_bt, sorted=True, stream=True)

    gi_novar = [[c, int(s), int(e), int(cov)] for c, s, e, n, cov in gi_gv]
    return gi_novar


def get_seq(fasta_index, intervals, coverage, length):
    '''
    Extract the sequences covered by all/most isoforms
    '''    
    
    seg_lst = []
    for seg in intervals:
        #drop segments not covered by all/almost all isoforms
        if seg[3] >= coverage:
            #drop those that are shorter than indicated length
            seg_len = seg[2] - seg[1]
            if seg_len >= length:
                #extract whole segment sequence from the fasta file (*BED-like coordinates*)
                seg_lst.append([seg, str(fasta_index[seg[0]].seq[seg[1]:seg[2]])])
    seg_len = 0
    for e in seg_lst:
        seg_len += len(e[1])
    print('Processing {!s} nucleotides in {!s} segments.'.format(seg_len, len(seg_lst)))
    return seg_lst


def gen_seq(coord, sequence, length, strand, GC_lims, index=0):
    # generate N long nucleotide streches of CDS, not containing any RepeatMasker marked low complexity regions
    
    while index <= len(sequence) - length:
        seq = sequence[index:index + length]
        # generate reverse complement, transcribed sequences, depending on gene strand
        if strand == '+':
            rna_seq = transcribe(revcomp(seq))
        else:
            rna_seq = transcribe(seq)
        s_coords = [coord[0],coord[1] + index, coord[1] + index + length]
        index +=1
        # drop the sequence when outside of the GC content limits
        gc_content = round(gc(rna_seq), 2)
        if gc_content < GC_lims[0] or gc_content > GC_lims[1]:
            continue
        # do not generate sequences with soft masked nucleotides
        if rna_seq.isupper():
            yield [s_coords, rna_seq, gc_content]
    return


def count_seq(segments, length, GC_lims):
    # count the number of sequences for processing
    
    GC_high = 0
    GC_low = 0
    sm_droped = 0
    good = 0
    for coord, sequence in segments:
        index = 0
        while index <= len(sequence) - length:
            seq = sequence[index:index + length]
            index +=1
            # count the sequence when outside of the GC content limits
            gc_content = round(gc(seq), 2)
            if gc_content < GC_lims[0]:
                GC_low += 1
                continue
            if gc_content > GC_lims[1]:
                GC_high += 1
                continue
            # count sequences with soft masked nucleotides
            if seq.isupper():
                good += 1
            else:
                sm_droped += 1
    total = GC_high + GC_low + sm_droped + good
    print('Valid sequences: {} / {} (GC_low = {}, GC_high = {}, sm = {})'.format(good,
                                                                                 total, 
                                                                                 GC_low, 
                                                                                 GC_high, 
                                                                                 sm_droped))
    return good


def run_RNAcofold(seq):
    # Run RNAcofold to estimate the duplex folding change in free energy
    
    in_str = seq + '&' + seq
    params = ['-a0',
              '-d2',
              '--noLP',
              '--noPS',
              '--output-format=D',
              '--csv-noheader']
    
    proc = subprocess.run(['RNAcofold'] + params,
                          stdout=subprocess.PIPE,
                          stderr=subprocess.PIPE,
                          input=in_str,
                          encoding='ascii')
    if proc.returncode != 0:
        print(proc.stderr)
        proc.check_returncode()
    else:
        result = proc.stdout.split(',')
        en = {'dG':Decimal(result[7]),
              'AB':Decimal(result[8]),
              'AA':Decimal(result[9]),
              'BB':Decimal(result[10]),
              'A':Decimal(result[11]),
              'B':Decimal(result[12])}
        return en

    
def get_tmpfs():
    '''
    Helper function to to pick tmpfs folder and check if it exists on different OSes.
    
    Might need to be developed further.
    '''
    
    tmp_dir = os.path.join('/dev', 'shm')
    if not os.path.exists(tmp_dir):
        tmp_dir = os.path.join('/run', 'shm')
    assert os.path.exists(tmp_dir)
    return tmp_dir
    
    
def run_RNAfold(transcript_id, transcript_seq):
    
    params = ['-p1',
              '-d2',
              '--noLP',
              '--noPS']
    
    in_str = '>{}\n'.format(transcript_id) + transcript_seq
    
    tmp_dir = get_tmpfs()
    
    proc1 = subprocess.run(['RNAfold'] + params,
                           stdout=subprocess.DEVNULL,
                           stderr=subprocess.PIPE,
                           input=in_str,
                           encoding='ascii',
                           cwd=tmp_dir)
    if proc1.returncode != 0:
        print(proc1.stderr)
        proc1.check_returncode()
    
    out_fn = os.path.join(tmp_dir, transcript_id + '_dp.ps')
    assert os.path.exists(out_fn)
    
    proc2 = subprocess.run(['mountain.pl', out_fn],
                           stderr=subprocess.PIPE,
                           stdout=subprocess.PIPE,
                           cwd=tmp_dir)
    if proc2.returncode != 0:
        print(proc2.stderr)
        proc2.check_returncode()
    else:
        data = proc2.stdout.decode('ascii').split('\n')
        elements = [e.split() for e in data]
        mark = []
        for i, position in enumerate(elements):
                if position != [] and position[0] == '&':
                    mark.append(i)
        energy = [Decimal(v) for k, v in elements[mark[1] + 1 :-1]]
        return energy

    
def get_trans(database, fasta_index, gene):
    '''
    Obtain transcript sequences
    
    TODO: with biopython this is slow, as it accesses the fasta file from the begining of each record
    for each exon of every transcript and needs a future rewrite
    
    For speed, use:
    blastdbcmd -entry [tr.id, tr['transcript_version']] -db danRer_e91_allrna
    '''
    
    for transcript in db.children(gene, featuretype='transcript', order_by='start'):
        if transcript['transcript_biotype'][0] == 'protein_coding':
            trans_seq = None
            for exon in db.children(transcript, featuretype='exon', order_by='start'):
                if not trans_seq:
                    trans_seq = str(fasta_index[transcript.chrom].seq[exon.start - 1:exon.end])
                else:
                    trans_seq += str(fasta_index[transcript.chrom].seq[exon.start - 1:exon.end])
            if transcript.strand == '-':
                trans_seq = revcomp(trans_seq)
            yield transcript.id, trans_seq
    return


def pick_transcript(database, fasta_index, gene=None, eid=None):
    '''
    Pick only the longest from protein coding, preferably havana/ensembl+havana annotated transcripts 
    and return it's id and sequence;
    if ensembl id is provided by user return sequence of that particular one.
    '''
    
    if not eid:
        print('No transcript Ensemble ID specified.\nPicking the longest, \
preferably HAVANA annotated transcript for free energy calculations.')
        max_len = 0
        max_id = None
        max_exons = None
        max_source = None
        for transcript in db.children(gene, featuretype='transcript', order_by='start'):
            if transcript['transcript_biotype'][0] == 'protein_coding':
                t_len = 0
                t_id = transcript.id
                t_exo = []
                t_sou = transcript['transcript_source'][0]
                for exon in db.children(transcript, featuretype='exon', order_by='start'):
                    # calculate the transcript lenght from it's exons' lenght, store the exons' coords, 
                    # do not extract the sequence yet
                    t_len += exon.end - exon.start + 1
                    t_exo.append((exon.start - 1, exon.end))
                if t_len > max_len and ((max_source == 'ensembl' or max_source == None) or\
                (t_sou == 'havana' or t_sou == 'ensembl_havana')):
                    max_len = t_len
                    max_id = t_id
                    max_exons = t_exo
                    max_source = t_sou
                elif max_source == 'ensembl' and (t_sou == 'havana' or t_sou == 'ensembl_havana'):
                    # prefer havana annotated transcripts
                    max_len = t_len
                    max_id = t_id
                    max_exons = t_exo
                    max_source = t_sou
                elif t_len == max_len and t_sou != max_source:
                    # if by chance two transcripts are of equal lenght pick the one with merged annotation
                    # if both have the same annotation source keep the first one
                    if t_sou == 'ensembl_havana':
                        max_len = t_len
                        max_id = t_id
                        max_exons = t_exo
                        max_source = t_sou
                elif t_len == max_len:
                    print('W: Discarding transcript {}, source: {}, of equal length: {!s} to {}'.format(t_id, 
                                                                                                       t_sou, 
                                                                                                       t_len,
                                                                                                       max_id))
        print('Target: {}, source: {}, length: {!s}'.format(max_id, max_source, max_len))
        max_seq = None
        for exon_coord in max_exons:
            if not max_seq:
                max_seq = str(fasta_index[gene.chrom].seq[exon_coord[0]:exon_coord[1]])
            else:
                max_seq += str(fasta_index[gene.chrom].seq[exon_coord[0]:exon_coord[1]])
        if gene.strand == '-':
            max_seq = revcomp(max_seq)
        return max_id, max_seq
    else:
        transcript = db[eid]
        trans_seq = None
        for exon in db.children(transcript, featuretype='exon', order_by='start'):
            if not trans_seq:
                trans_seq = str(fasta_index[transcript.chrom].seq[exon.start - 1:exon.end])
            else:
                trans_seq += str(fasta_index[transcript.chrom].seq[exon.start - 1:exon.end])
        if transcript.strand == '-':
            trans_seq = revcomp(trans_seq)
        print('Target: {}, source: {}, length: {!s}'.format(transcript.id, transcript['transcript_source'][0], len(trans_seq)))
        return transcript.id, trans_seq
    
    
def create_negseqidlst(database, gene_id=None, transcript_id=None):
    '''
    Create temporary file with transcript id's to exclude from blast searches.
    
    When transcript Id is given, list all transcripts of the parent gene.
    '''
    
    g_id = gene_id
    if transcript_id:
        g_id = database[transcript_id]['gene_id'][0]
    
    gene = database[g_id]
    
    ngsi = ''
    for i,f in enumerate(database.children(gene, featuretype='transcript')):
        if i == 0:
            ngsi += '.'.join([f.id, f['transcript_version'][0]])
        else:
            ngsi += '\n' + '.'.join([f.id, f['transcript_version'][0]])

    tmp_fn = g_id + '.balst.ngsi.lst'
    tmp_file = os.path.join(get_tmpfs(), tmp_fn)
    with open(tmp_file, 'w') as handle:
        handle.write(ngsi)
    return tmp_file


def blast_it(sequence, tmp_file=None):
    '''
    Run blastn on a sequence (wordsize 7).
    Return Bitscore, nident
    
    *Might* be faster to run on all sequences beforehand.
    '''
    
    assert isinstance(sequence, str)

    params = ['-task', 'blastn',
              '-word_size', '7',
              '-db', './blastdb/danRer_e91_allrna',
              '-outfmt', '6 bitscore nident',
              '-num_threads', '1',
              '-max_target_seqs', '1']
    if tmp_file:
        params += ['-negative_seqidlist', tmp_file]

    proc = subprocess.run(['blastn',
                           '-query', '-',
                           '-out', '-'] + params,
                          stdout=subprocess.PIPE,
                          stderr=subprocess.PIPE,
                          input=sequence,
                          encoding='ascii')

    if proc.returncode != 0:
        print(proc.stderr)
        proc.check_returncode()
    else:
        b_out = proc.stdout.split()
        if len(b_out) == 0:
            bitscore = -1
            nident = 0
            return bitscore, nident
        else:
            bitscore = float(b_out[0])
            nident = int(b_out[1])
            return bitscore, nident

        
def feed_fun(segments, length, strand, GC_lims, transcript_seq, entropy, tmp_path=None):
    for coordinates, sequence in segments:
        for result in gen_seq(coordinates, sequence, length,
                              strand, GC_lims, index=0):
            yield result + [transcript_seq] + [tmp_path] + [entropy]
    return


def processing_fun(input_list):
    coords, c_seq, gc_content, t_seq, ngsi_tmp, entropy = input_list
    # get the mean entropy of the target sequence
    # TODO: !!! this is far from perfect
    # t_seq.upper() required in case of working with user-input and targeting soft masked sequence 
    c_start = t_seq.upper().find(revcomp(revtranscribe(c_seq)))
    assert c_start != -1, 'Target sequence not found: {}'.format(revcomp(revtranscribe(c_seq)))
    c_pos_ent = entropy[c_start:c_start + len(c_seq)]
    c_mean_ent = sum(c_pos_ent) / len(c_pos_ent)
    # estimate also the change in free energy of monomer binding to itself,
    energy = run_RNAcofold(c_seq)
    dG_AA = energy['dG']
    G_A = energy['A']
    # blast the sequence against the Danio rerio RNA database
    bs, ni = blast_it(c_seq, ngsi_tmp)
    return coords, c_seq, gc_content, c_mean_ent, dG_AA, G_A, bs, ni
    
    
def estimate_energy_all(database, fasta_index, gene, intervals, coverage, length, GC_lims, proc=1):
    '''
    Estimate free energy change for each segment for all transcripts- very inefficient
    for 3 isoforms and ~500 segments takes ~10-12h on 4 procesors/ 7 threads.
    '''
    
    pool = Pool(proc)
    strand = gene.strand
    segments = get_seq(fasta_index, intervals, coverage, length)
    result_dict = defaultdict(list)
    for transcript_id, transcript_seq in get_trans(database, fasta_index, gene):
        iterator = feed_fun(segments, length, strand, GC_lims, transcript_seq)
        for result in pool.imap_unordered(processing_fun, iterator,
                                          chunksize=1):
            coords, c_seq, gc_content, dG, dG_AA = result
            if coords not in result_dict[coords[1]]:
                result_dict[coords[1]].append(coords)
            if c_seq not in result_dict[coords[1]]:
                result_dict[coords[1]].append(c_seq)
            if gc_content not in result_dict[coords[1]]:
                result_dict[coords[1]].append(gc_content)
            if ('AA', dG_AA) not in result_dict[coords[1]]:
                result_dict[coords[1]].append(('AA', dG_AA))
            result_dict[coords[1]].append((transcript_id, dG))
    return result_dict


def estimate_energy(database, fasta_index, gene, intervals, coverage, length, GC_lims, ensembl_id=None, proc=1):
    '''
    Estimate free energy change for a single transcript
    '''
    
    pool = Pool(proc)
    strand = gene.strand
    segments = get_seq(fasta_index, intervals, coverage, length)
    total = count_seq(segments, length, GC_lims)
    ngsi_tmp = create_negseqidlst(database, gene_id=gene.id)
    result_list = []
    transcript_id, transcript_seq = pick_transcript(database, fasta_index, gene, ensembl_id)
    # get the positional entropy for the whole transcript
    entropy = run_RNAfold(transcript_id, transcript_seq)
    iterator = feed_fun(segments, length, strand, GC_lims, transcript_seq, entropy, ngsi_tmp)
    done = 0
    for result in pool.imap_unordered(processing_fun, iterator,
                                      chunksize=1):
        done += 1
        progress = str(done) + '/' + str(total)
        percent = str(round(100 * Decimal(done) / Decimal(total), 1)) + '%'
        out = '{} ({})'.format(progress, percent)
        print(out, end='', flush=True)
        print('\r', end='')
        result_list.append(result)
    # clean up
    # blast negative seqid list file
    os.remove(ngsi_tmp)
    # RNAfold dot plot PS file
    os.remove(os.path.join(get_tmpfs(), transcript_id + '_dp.ps'))
    return result_list


def processing_input(string):
    '''
    Process the input sequence and return the elements needed for further steps.
    '''
    
    # quick checks
    assert isinstance(string, str), 'Input not a srting.'
    # take care of whitespaces
    sequence = string.strip()
    assert sequence.upper().startswith(('>', 'A', 'G', 'C', 'T')), 'Input not a DNA sequence in FASTA format or PLAIN.'
    
    # if fasta, get the sequence id from the header
    if sequence.startswith('>'):
        id_line, seq = sequence.split('\n', maxsplit=1)
        seq_id = id_line.lstrip('>').split()[0]
    else:
        seq_id = 'plain'
        seq = sequence
    
    # join all the lines if present, also taking care of whitespaces
    seq = ''.join(l.strip() for l in seq.split('\n'))
                          
    return seq_id, seq


def check_segid(database, input_id):
    '''
    Helper function to check if the sequence ID is valid for negative seqid list file creation.
    
    Will have to be developed further.
    '''
    
    try:
        feature = database[input_id]
    except gffutils.FeatureNotFoundError:
        print('Feature \'{}\' not found in the database.'.format(input_id))
        return False
    else:
        if feature.featuretype == 'transcript':
            return True
        else:
            print('\'{}\' is not a valid Transcript ID.'.format(input_id))
            return False
    

def estimate_energy_input(input_sequence, length, GC_lims, strand='+', database=None, fasta_index=None, proc=1):
    '''
    Estimate free energy change for a single transcript.
    
    Parameters:
    
    strand- use if input is genomic sequence
    database; fasta_index - must be provided if database Transcript ID lookup is required
    '''
    
    pool = Pool(proc)
    input_id, input_seq = processing_input(input_sequence)
    # assume that for the input the strand is always the Watson one
    segment = [[['.', 0, len(input_seq)], input_seq]]
    total = count_seq(segment, length, GC_lims)
 
    ngsi_tmp = None
    # check if the input ID is in the database and if it is a valid transcript ID, if yes use it to create
    # negative segid list for blast and fold this particular transcript for energy estimations
    if database and fasta_index and check_segid(database, input_id):
        ngsi_tmp = create_negseqidlst(database, transcript_id=input_id)
        transcript_id, transcript_seq = pick_transcript(database, fasta_index, eid=input_id)
    else:
        print('Using the provided sequence for free energy calculation.')
        transcript_id, transcript_seq = input_id, input_seq

    result_list = []
    # get the positional entropy for the whole sequence/transcript
    entropy = run_RNAfold(transcript_id, transcript_seq)
    iterator = feed_fun(segment, length, strand, GC_lims, transcript_seq, entropy, ngsi_tmp)
    done = 0
    for result in pool.imap_unordered(processing_fun, iterator,
                                      chunksize=1):
        done += 1
        progress = str(done) + '/' + str(total)
        percent = str(round(100 * Decimal(done) / Decimal(total), 1)) + '%'
        out = '{} ({})'.format(progress, percent)
        print(out, end='', flush=True)
        print('\r', end='')
        result_list.append(result)
    # clean up
    # blast negative seqid list file
    if ngsi_tmp:
        os.remove(ngsi_tmp)
    # RNAfold dot plot PS file
    os.remove(os.path.join(get_tmpfs(), transcript_id + '_dp.ps'))
    return result_list


def main(database, fasta_index, var_db, target_lst, crRNA_lenght, GC_limit, n_threads, coverage_limit='max'):
    '''
    Main function to run the program.
    '''
    
    result = []
    for target in target_lst:
        print('\n---\n')
        gene = database[target]
        g_name = gene['gene_name'][0]
        print(g_name)
        gene_cov = get_cov(database, gene)
        cov_lims = coverage_limit
        if coverage_limit == 'max':
            cov_lims = gene_cov.max()
        if gene.strand == '+':
            plot_cov = gene_cov
        else:
            plot_cov = gene_cov[::-1]
        plt.plot(plot_cov, 'blue')
        plt.show()
        gen_int = sub_var(get_int(gene, gene_cov), var_db)
        output = estimate_energy(database,
                                 fasta_index,
                                 gene,
                                 gen_int,
                                 cov_lims,
                                 crRNA_len,
                                 GC_limit,
                                 proc=n_threads)

        fn = target + '.result.csv'
        print('\nWritting file {}.'.format(fn))
        with open(fn, 'w') as handle:
            for item in output:
                handle.write(','.join([str(e) for e in item]) + '\n')

        result.append((g_name, output))
        print('\nDone.')
    return result


def prepare_output(result, n_store=50, write=True):
    '''
    Prepare the data frames, score and store the output.
    '''
    
    final_dict = {}
    for name, output in result:

        labels = ['coords', 'seq', 'GC', 'ent', 'dG_AA', 'G_A', 'bitscore', 'nident']
        df = pd.DataFrame.from_records(output, columns=labels)
        df[['chr', 'start', 'end']] = pd.DataFrame([e for e in df['coords']])
        del df['coords']

        # reset index to match the start position (shifted by multiprocessing)
        df = df.sort_values('start').reset_index(drop=True)
        
        score = []
        df = df.sort_values('ent', ascending=False)
        points = {}
        point = 1
        for e in df['ent'].unique():
            points[e] = point
            point += 1
        for e in df['ent']:
            score.append(points[e])
        score
        df['score'] = score

        blast_score = []
        blast_points = {}
        df = df.sort_values('bitscore')
        num_uniq_bits = len(df['bitscore'].unique())
        point_step = Decimal(max(points.values())) / num_uniq_bits
        blast_point = point_step
        for e in df['bitscore'].unique():
            blast_points[e] = blast_point
            blast_point += point_step
        for e in df['bitscore']:
            blast_score.append(blast_points[e])
        df['score'] += blast_score

        df = df.sort_values('score')
        
        if n_store <= 0:
            final_dict[name] = df
        else:
            final_dict[name] = df.head(n_store)
    if write:
        for key in final_dict:
            xlsx_fn = key + '.csm.xlsx'
            print('Writing file {}.'.format(xlsx_fn))
            final_dict[key].to_excel(xlsx_fn)
    return final_dict


#TODO: things to consider when doing a rewrite
# -- minor gain would be to run RNAcofold -p instead of -a and not calculate the dG of AA complex formation

## round it to 2 decimals
#    dG_AA = round(cofold_dict['AA'] - cofold_dict['A'] * 2, 2)

---

**ENTER AN ENSEMBLE GENE ID HERE**

In [ ]:
gene = db['ENSDARG00000008454']

Create genomic sequence coverage by **CDS** (without STOP codon).

In [ ]:
gene_cov = get_cov(db, gene)

For ploting change start coordinates to local and reverse the coverage if needed

In [ ]:
if gene.strand == '+':
    plot_cov = gene_cov
else:
    plot_cov = gene_cov[::-1]

Plotting

In [ ]:
plt.plot(plot_cov, 'blue')
plt.show()

Get the genomic intervals for sequene retrieval. Subtract the varable sequences comming from dbSNP.

In [ ]:
gen_int = sub_var(get_int(gene,gene_cov), gvf_fn)

---

Define search parameters.

In [ ]:
crRNA_len = 36
GC_limit = (40,60)
# number of processors to use
n_threads = 7

In [ ]:
targets = ['ENSDARG00000008454', 
           'ENSDARG00000012389',
           'ENSDARG00000101576',
           'ENSDARG00000014373',
           'ENSDARG00000035095',
           'ENSDARG00000017821']

In [ ]:
out = main(db, fa_db, gvf_fn, targets, crRNA_len, GC_limit, n_threads, coverage_limit='max')

In [ ]:
formated = prepare_output(out, n_store=0, write=False)

In [ ]:
formated['ta']

---

Mode for working on user given sequence

In [211]:
egfp_seq = 'aggatccaccggtcgccaccatggtgagcaagggcgaggagctgttcaccggggtggtgcccatcctggtcgagctggacggcgacgtaaacggccacaagttcagcgtgtccggcgagggcgagggcgatgccacctacggcaagctgaccctgaagttcatctgcaccaccggcaagctgcccgtgccctggcccaccctcgtgaccaccctgacctacggcgtgcagtgcttcagccgctaccccgaccacatgaagcagcacgacttcttcaagtccgccatgcccgaaggctacgtccaggagcgcaccatcttcttcaaggacgacggcaactacaagacccgcgccgaggtgaagttcgagggcgacaccctggtgaaccgcatcgagctgaagggcatcgacttcaaggaggacggcaacatcctggggcacaagctggagtacaactacaacagccacaacgtctatatcatggccgacaagcagaagaacggcatcaaggtgaacttcaagatccgccacaacatcgaggacggcagcgtgcagctcgccgaccactaccagcagaacacccccatcggcgacggccccgtgctgctgcccgacaaccactacctgagcacccagtccgccctgagcaaagaccccaacgagaagcgcgatcacatggtcctgctggagttcgtgaccgccgccgggatcactctcggcatggacgagctgtacaagtaaagcggccgc'

In [212]:
egfp_res = estimate_energy_input(egfp_seq.upper(), 36, (0,100), proc=7)

Valid sequences: 714 / 714 (GC_low = 0, GC_high = 0, sm = 0)
Using the provided sequence for free energy calculation.


In [213]:
egfp_out = prepare_output([('eGFP', egfp_res)], n_store=0)

Writing file eGFP.csm.xlsx.


In [214]:
egfp_out['eGFP'].head()

,seq,GC,ent,dG_AA,G_A,bitscore,nident,chr,start,end,score
355,AUGCGGUUCACCAGGGUGUCGCCCUCGAACUUCACC,61.11,1.411554944444444444444444444,-10.52,-11.306073,28.3,18,.,355,391,166.8
170,CACGAGGGUGGGCCAGGGCACGGGCAGCUUGCCGGU,75.00,1.345367777777777777777777778,-9.36,-10.663815,28.3,18,.,170,206,188.8
169,ACGAGGGUGGGCCAGGGCACGGGCAGCUUGCCGGUG,75.00,1.3156325,-7.69,-10.650033,28.3,18,.,169,205,199.8
168,CGAGGGUGGGCCAGGGCACGGGCAGCUUGCCGGUGG,77.78,1.262398333333333333333333333,-7.76,-11.742355,28.3,17,.,168,204,224.8
649,AGCAGGACCAUGUGAUCGCGCUUCUCGUUGGGGUCU,58.33,1.261723222222222222222222222,-9.56,-7.146165,28.3,18,.,649,685,225.8


---